In [16]:
from hera.shared import global_config
from hera.auth import ArgoCLITokenGenerator

global_config.host = "https://localhost:2746"
global_config.token = ArgoCLITokenGenerator
global_config.verify_ssl = False

## Kubeflow 1.2 - compiles argo workflow manifests

In [17]:
import kfp
print(kfp.__version__)
def flip_coin():
    return kfp.dsl.ContainerOp(
        name='Flip a coin',
        image='python:alpine3.6',
        command=['python', '-c', """
import random
res = "heads" if random.randint(0, 1) == 0 else "tails"
with open('/output', 'w') as f:
    f.write(res)        
        """],
        file_outputs={'output': '/output'}
    )
    
def heads():
    return kfp.dsl.ContainerOp(name='Heads', image="alpine:3.6", command=["sh", "-c", 'echo "it was heads"'])

def tails():
    return kfp.dsl.ContainerOp(name='Tails', image="alpine:3.6", command=["sh", "-c", 'echo "it was tails"'])

@kfp.dsl.pipeline(name='Coin-flip', description='Flip a coin')
def coin_flip_pipeline():
    flip = flip_coin()
    with kfp.dsl.Condition(flip.output == 'heads'):
        heads()
    with kfp.dsl.Condition(flip.output == 'tails'):
        tails()

1.2.0


In [20]:
kfp.compiler.Compiler().compile(coin_flip_pipeline,'kfp-pipeline.yaml')

c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\kfp\dsl\_container_op.py:1032: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  warnings.warn(


## Kubeflow 1.2 - submit pipeline

In [21]:
from hera.workflows import Workflow

with open('kfp-pipeline.yaml','r') as kubeflow_pipeline_manifest:
    kubeflow_pipeline = Workflow.from_yaml(kubeflow_pipeline_manifest)

kubeflow_pipeline.namespace = "argo"
kubeflow_pipeline.create()

c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Workflow(api_version=None, kind=None, metadata=ObjectMeta(annotations={'pipelines.kubeflow.org/kfp_sdk_version': '1.2.0', 'pipelines.kubeflow.org/pipeline_compilation_time': '2024-04-30T23:06:24.400587', 'pipelines.kubeflow.org/pipeline_spec': '{"description": "Flip a coin", "name": "Coin-flip"}'}, cluster_name=None, creation_timestamp=Time(__root__=datetime.datetime(2024, 4, 30, 22, 7, 1, tzinfo=datetime.timezone.utc)), deletion_grace_period_seconds=None, deletion_timestamp=None, finalizers=None, generate_name='coin-flip-', generation=1, labels={'pipelines.kubeflow.org/kfp_sdk_version': '1.2.0', 'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, managed_fields=[ManagedFieldsEntry(api_version='argoproj.io/v1alpha1', fields_type='FieldsV1', fields_v1=FieldsV1(), manager='argo', operation='Update', subresource=None, time=Time(__root__=datetime.datetime(2024, 4, 30, 22, 7, 1, tzinfo=datetime.timezone.utc)))], name='coin-flip-k6dcw', namespace='argo', owner_referen

## Hera - compiles workflow manifest

In [22]:
from hera.workflows import Workflow, DAG, script, Artifact, Parameter

@script()
def flip_coin():
    import random
    res = "heads" if random.randint(0, 1) == 0 else "tails"
    print(res)
        
@script()
def heads():
    print("it was heads")
    
@script()
def tails():
    print("it was tail")

with Workflow(generate_name="flip-a-coin-", entrypoint="Coin-flip",namespace="argo") as w:
    with DAG(name="Coin-flip"):
        fc = flip_coin(name="Flip-a-coin")
        
        heads(name="Heads").on_other_result(fc, "heads") # accesses the stdout from the fc task and checks if its "heads"
        
        tails(name="Tails").on_other_result(fc, "tails") # accesses the stdout from the fc task and checks if its "tails"

In [23]:
import yaml

with open('hera-pipeline.yaml','w') as pipeline_file:
    yaml.dump(w.to_yaml(),pipeline_file)

# Hera - submit pipeline

In [24]:
w.create()

c:\Users\bettmensch\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Workflow(api_version=None, kind=None, metadata=ObjectMeta(annotations=None, cluster_name=None, creation_timestamp=Time(__root__=datetime.datetime(2024, 4, 30, 22, 9, 9, tzinfo=datetime.timezone.utc)), deletion_grace_period_seconds=None, deletion_timestamp=None, finalizers=None, generate_name='flip-a-coin-', generation=1, labels={'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, managed_fields=[ManagedFieldsEntry(api_version='argoproj.io/v1alpha1', fields_type='FieldsV1', fields_v1=FieldsV1(), manager='argo', operation='Update', subresource=None, time=Time(__root__=datetime.datetime(2024, 4, 30, 22, 9, 9, tzinfo=datetime.timezone.utc)))], name='flip-a-coin-6mzkt', namespace='argo', owner_references=None, resource_version='41525', self_link=None, uid='8aeb4732-1890-49b3-a3cc-7a203323e41b'), spec=WorkflowSpec(active_deadline_seconds=None, affinity=None, archive_logs=None, arguments=Arguments(artifacts=None, parameters=None), artifact_gc=None, artifact_repository_